In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.models.keyedvectors as word2vec
import gc

Using TensorFlow backend.
C:\Users\E\Anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\E\Anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\E\Anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\E\Anaconda3\envs\learn-env\lib\site-pa

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
embed_size=0

In [3]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [4]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [5]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [6]:
embed_size = 200
embeddings_index = dict()
#Transfer the embedding weights into a dictionary by iterating through every line of the file.
f = open('glove.twitter.27B.200d.txt', encoding="utf8")
for line in f:
    #split up line into an indexed array
    values = line.split()
    #first index is word
    word = values[0]
    #store the rest of the values in the array as a new array
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs #50 dimensions
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [7]:
embeddings_index = {k:v for k,v in embeddings_index.items() if v.shape[0]==200}

In [8]:
gc.collect()
#We get the mean and standard deviation of the embedding weights so that we could maintain the 
#same statistics for the rest of our own random generated weights. 
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()

nb_words = len(tokenizer.word_index)
#We are going to set the embedding size to the pretrained dimension as we are replicating it.
#the size will be Number of Words in Vocab X Embedding Size
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
gc.collect()
#With the newly created embedding matrix, we'll fill it up with the words that we have in both 
#our own dictionary and loaded pretrained embedding. 
embeddedCount = 0
for word, i in tokenizer.word_index.items():
    i-=1
    #then we see if this word is in glove's dictionary, if yes, get the corresponding weights
    embedding_vector = embeddings_index.get(word)
    #and store inside the embedding matrix that we will train later on.
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
        embeddedCount+=1
print('total embedded:',embeddedCount,'common words')

del(embeddings_index)
gc.collect()

total embedded: 81610 common words


0

In [9]:
embedding_matrix.shape

(210337, 200)

In [10]:
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier

In [11]:
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)

In [12]:
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)

In [13]:
x = GlobalMaxPool1D()(x)

In [14]:
x = Dropout(0.1)(x)

In [15]:
x = Dense(50, activation="relu")(x)

In [16]:
x = Dropout(0.1)(x)

In [17]:
x = Dense(6, activation="sigmoid")(x)

In [18]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [19]:
batch_size = 32
epochs = 4
hist = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143613/143613 [==============================] - 2383s 17ms/step - loss: 0.0778 - acc: 0.9757 - val_loss: 0.0574 - val_acc: 0.9798
Epoch 2/4
143613/143613 [==============================] - 2406s 17ms/step - loss: 0.0556 - acc: 0.9809 - val_loss: 0.0576 - val_acc: 0.9804
Epoch 3/4
143613/143613 [==============================] - 2437s 17ms/step - loss: 0.0507 - acc: 0.9820 - val_loss: 0.0511 - val_acc: 0.9815
Epoch 4/4
143613/143613 [==============================] - 2439s 17ms/step - loss: 0.0474 - acc: 0.9829 - val_loss: 0.0503 - val_acc: 0.9822


## Credit

Since I am new at Deep Learning and just wanted to make an attempt on my computer, I need to give credit to this shared notebook for pretty much all of the code

https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge